# COCO Image Tiler

Tile larger COCO annotated images into smaller COCO annotated images

This involves cutting up the images and fixing the COCO annotations so that they align with the new images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import io
from PIL import Image

import os
print(f"Current OS: {os.name}") 

## Utils

In [ ]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations["image/height"]
    width = annotations["image/width"]
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for j in range(len(annotations['image/object/class/label'])):
        x1 = annotations["image/object/bbox/xmin"][j] * width
        y1 = annotations["image/object/bbox/ymin"][j] * height
        x2 = annotations["image/object/bbox/xmax"][j] * width - x1
        y2 = annotations["image/object/bbox/ymax"][j] * height - y1
        
        category_num = annotations["image/object/class/label"][j]-1
        label = categories[category_num]["name"]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

In [ ]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels2(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations['image']['height']
    width = annotations['image']['width']
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for row in annotations['annotations']:
        x1, y1, x2, y2 = row['bbox']
        
        category_num = row['category_id']
        label = categories[category_num]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

In [ ]:
def counter(reset = None):
    count=0
    def increment():
        nonlocal count
        nonlocal reset
        if reset == None:
            count += 1
        else:
            count = reset
            reset = None
        return count
    return increment

In [ ]:
def filename_append(filename, append_text):
    old_file = os.path.splitext(filename)
    new_file_name = old_file[0]+'_'+str(append_text)+old_file[1]
    return new_file_name

## COCO JSON structure

{
    
    "info": {
        "year": 2022,
        "version": "1.0",
        "description": "VIA project exported to COCO format using VGG Image Annotator (http://www.robots.ox.ac.uk/~vgg/software/via/)",
        "contributor": "",
        "url": "http://www.robots.ox.ac.uk/~vgg/software/via/",
        "date_created": "Sat Jul 30 2022 19:22:25 GMT+1200 (New Zealand Standard Time)"
    },

    "images": [
        {
            "id": 23,
            "width": 6000,
            "height": 4000,
            "file_name": "Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "license": 0,
            "flickr_url": "./Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "coco_url": "./Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "date_captured": ""
        },
    ],
    
    "annotations": [
        {
            "segmentation": [
                [
                    1006,
                    1384,
                    1084,
                    1384,
                    1084,
                    1596,
                    1006,
                    1596
                ]
            ],
            "area": 16536,
            "bbox": [
                1006,
                1384,
                78,
                212
            ],
            "iscrowd": 0,
            "id": 1,
            "image_id": 1,
            "category_id": 3
        },
    ],
    
    
    "licenses": [
        {
            "id": 0,
            "name": "Unknown License",
            "url": ""
        }
    ],
    
    "categories": [
        {
            "supercategory": "Damage",
            "id": 1,
            "name": "None"
        },
    ]    
}

### Get COCO Info

In [ ]:
# Define a list of folder paths to be created (if needed) and used later
paths = {
    #'IMAGE_SOURCE_PATH':os.path.join('images'),
    'IMAGE_SOURCE_PATH':os.path.join('E:', os.sep,'Users','Vince','Datasets','NZRC','Gordon','CycloneGitaRawImagery','ML4DR_20210910_01'),
    
    # bounding box annotation
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
}


In [ ]:
root_dir = os.path.join('E:', os.sep ,'Users', 'Vince', 'Datasets', 'NZRC', 'Gordon', 'CycloneGitaRawImagery')
tfrecords_dir = os.path.join(root_dir, "ML4DR_20210910_01/tfrecords/val2017")
images_dir = os.path.join(root_dir, "ML4DR_20210910_01")
annotations_dir = os.path.join(root_dir, "ML4DR_20210910_01")
annotation_file = os.path.join(annotations_dir, "ML4DR_20210910_01_coco.json")
paths['IMAGE_PATH'] = images_dir


In [ ]:
#Load COCO file information

import json

#Load file info from COCO file
with open(annotation_file, "r") as f:
    coco_file_info = json.load(f)["info"]

#Load image refs from COCO file
with open(annotation_file, "r") as f:
    coco_images = json.load(f)["images"]

#Load annotations from COCO file
with open(annotation_file, "r") as f:
    coco_annotations = json.load(f)["annotations"]

#Load licenses from COCO file
with open(annotation_file, "r") as f:
    coco_licenses = json.load(f)["licenses"]

#Load categories from COCO file
with open(annotation_file, "r") as f:
    coco_categories = json.load(f)["categories"]

In [ ]:
print(type(coco_file_info))
print(type(coco_images))
print(type(coco_annotations))
print(type(coco_licenses))
print(type(coco_categories))

In [ ]:
#Rearrange COCO file info into a form that can be used to create the TF Record file
# this requires the annotation info for each image be associated to the image info
# and it requires a category dictionary to convert from category_id to category_text

#Work through coco-images
# Extract the image attributes
# For each image get a list of annotations
# Add the annotations list to the image

imageInfoList = []
for coco_image_info in coco_images:
    annotationsList = []
    for annotation in coco_annotations:
        if coco_image_info['id'] == annotation['image_id']:
            annotationsList.append(annotation)
    if len(annotationsList)>0:
        image_info={}
        image_info['image'] = coco_image_info
        image_info['annotations'] = annotationsList
        imageInfoList.append(image_info)
        
categoriesDict = {}
labels = []
for cat in coco_categories:
    labels.append({'name':cat['name'], 'id':cat['id']})
    categoriesDict[cat['id']] = cat['name']

In [ ]:
print(type(imageInfoList))
print(type(image_info))
print(type(categoriesDict))
print(type(labels))
print(categoriesDict)

### Save COCO Info

In [ ]:
# Recreate the COCO file with the new info (final step)

#Create COCO info structiore
COCO_info = {}

COCO_info["info"] =coco_file_info
COCO_info["images"] = coco_images
COCO_info["annotations"] = coco_annotations
COCO_info["licenses"] = coco_licenses
COCO_info["categories"] = coco_categories

#Save COCO info to a file
#????????????


### Process COCO info

In [ ]:
# Extract bboxes that are within the crop_box
# and adjust their coordinates to fit within the cropped image

def get_bboxes(crop_box, image_info, image_id):
    adj_annotations = []
    for a in image_info['annotations']: #Iterate through the annotations associated with the image
        bbox_x1, bbox_y1, bbox_x2, bbox_y2 = a['bbox']
        bbox_x2 += bbox_x1
        bbox_y2 += bbox_y1
        crop_x1, crop_y1, crop_x2, crop_y2 = crop_box
        
        b = {} # create a new dict to copy the values into
        
        # If top-left bbox corner is inside the crop box
        if crop_x1 < bbox_x1 and crop_y1 < bbox_y1:
            # If bottom-right bbox corner is inside the crop box
            if bbox_x2 < crop_x2 and bbox_y2 < crop_y2:
                # the bbox is inside the copy box
                
                # adjust the bbox coords to be inside the cropped image
                adj_bbox = ([a['bbox'][0]-crop_x1, a['bbox'][1]-crop_y1, a['bbox'][2], a['bbox'][3]])
                adj_segmentation = []
                for s in a['segmentation']:
                    adj_segment = []
                    for i in range(0, len(s), 2):
                        s_x, s_y = s[i: i+2].copy()
                        s_x = s_x - crop_x1
                        s_y = s_y - crop_y1
                        adj_segment.append(s_x)
                        adj_segment.append(s_y)
                    adj_segmentation.append(adj_segment)

                b['segmentation'] = adj_segmentation
                b['area'] = a['area']#.copy()
                b['bbox'] = adj_bbox
                b['iscrowd'] = a['iscrowd']#.copy()
                b['id'] = annotation_counter()
                b['image_id'] = image_id
                b['category_id'] = a['category_id']#.copy()

                adj_annotations.append(b)
    
    return adj_annotations


In [ ]:
image_counter = counter(1)
annotation_counter = counter(1)

In [ ]:
# Display an imageInfo record and the image

#imageInfo = imageInfoList[2].copy()
imageInfo = imageInfoList[2].copy()
image_path = paths['IMAGE_SOURCE_PATH']

with tf.io.gfile.GFile(os.path.join(image_path, '{}'.format(imageInfo['image']['file_name'])), 'rb') as fid:
    encoded_jpg = fid.read()
encoded_jpg_io = io.BytesIO(encoded_jpg)
image = Image.open(encoded_jpg_io)

print('type(imageInfo) =', type(imageInfo))
print('image_path =', image_path)
print('file_name =', imageInfo['image']['file_name'])
print('type(encoded_jpg) =', type(encoded_jpg))
print('type(encoded_jpg_io) =', type(encoded_jpg_io))
print('type(image) =', type(image))
print('imageInfo:')
#print(imageInfo)

#plt_image_with_labels2(image, imageInfo, coco_categories)

In [ ]:
# Extract image tiles
import numpy as np
import matplotlib.pyplot as plt
import json

crop_size = (1000, 1000)
crop_overlap = 200
image_size = image.size
print('master_image_size =', image_size)

img_count = 0
new_imageInfo = imageInfo.copy()
new_coco_image = {}
new_coco_images = []
new_coco_annotations = []

for x in range(0, image.size[0]-crop_overlap, crop_size[0]-crop_overlap): #(Start, End, Step)
    if x >= image_size[0]-crop_size[0]:
        x = image_size[0]-crop_size[0]
    for y in range(0, image.size[1]-crop_overlap, crop_size[1]-crop_overlap): #(Start, End, Step)
        if y >= image_size[1]-crop_size[1]:
            y = image_size[1]-crop_size[1]
        img_count += 1
        new_image_id = image_counter() #imageInfo['image']['id'] + img_count
        new_coco_image = new_imageInfo['image'].copy()
        new_coco_image['id'] = new_image_id
        new_coco_image['width'], new_imageInfo['image']['height'] = crop_size
        crop_box = (x, y, x+crop_size[0], y+crop_size[1])
        adj_annotations = get_bboxes(crop_box, imageInfo, new_image_id)
        
        #Need a function to insert the img_count into the file name
        new_coco_image['file_name'] = filename_append(imageInfo['image']['file_name'], str(new_image_id))
        new_coco_image['flickr_url'] = filename_append(imageInfo['image']['flickr_url'], str(new_image_id))
        new_coco_image['coco_url'] = filename_append(imageInfo['image']['coco_url'], str(new_image_id))

        new_coco_images.append(new_coco_image)
        if len(adj_annotations)>0:
            new_coco_annotations.extend(adj_annotations)

        print('new_image_id =', new_image_id)
        print('crop_box =', crop_box)
        
        filename = os.path.join('E:/Users/Vince/Datasets/NZRC/Gordon/CycloneGitaRawImagery/ML4DR_20210910_01/tiles/', new_coco_image['file_name'])
        print('filename', filename)
        image.crop(crop_box).save(filename)

        t={}
        t['annotations'] = adj_annotations
        t['image'] = new_coco_image
        plt_image_with_labels2(image.crop(crop_box), t, categoriesDict)

#write to json files
json_data = {}
json_data['images'] = new_coco_images
json_data['annotations'] = new_coco_annotations

filename = os.path.join('E:/Users/Vince/Datasets/NZRC/Gordon/CycloneGitaRawImagery/ML4DR_20210910_01/tiles/', 'ML4DR_20210910_01_coco_tiles.json')
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

#return new_imageInfo

In [ ]:
image_count

In [ ]:
new_coco_images

In [ ]:
new_coco_annotations